In [1]:
import pandas as pd
import random
import timeit
import time
import os

___
Functions to generate Target Indices
___

In [2]:
def checkGeneratedTargetIndex(indicesList, index, whatBack, level):
    result = True

    if whatBack == 3:
        for item in indicesList:
            for i in range(3 + whatBack - 1):
                if item + i == index or item - i == index:
                    result = False
    
    if whatBack == 1 and level != 1:
        for item in indicesList:
            for i in range(3 + whatBack - 1):
                if item + i == index or item - i == index:
                    result = False
    else:
        for item in indicesList:
            for i in range(3 + whatBack - 1):
                if item + i == index or item - i == index:
                    result = False
    return result


def createTargetIndices(lim, buffer, whatBack, level):
    indices = []

    if whatBack == 3:
        selector = random.randint(0, 2)
        if selector == 0:
            indices.append(26)
            indices.append(3)
        elif selector == 1:
            indices.append(25)
            indices.append(3)
        else:
            indices.append(24)
            indices.append(3)

    
    if whatBack == 1:
        if level != 1:
            selector = random.randint(0, 1)
            if selector == 0:
                indices.append(1)
            elif selector == 1:
                indices.append(15)

        else:
            selector = random.randint(0, 1)
            if selector == 0:
                indices.append(1)
            elif selector == 1:
                indices.append(2)

    
    
    starttime = timeit.default_timer()
    elapsed_time = 0

    while len(indices) != 5:

        if timeit.default_timer() - starttime > 5:
            break
        
        temp = random.randint(2 + (whatBack - 1),lim - 2)

        if whatBack == 3:
            temp = random.randint(4 + (whatBack - 1),lim - 2)
       
        if (temp in indices) or (checkGeneratedTargetIndex(indices, temp, whatBack, level) == False):
            continue
        
        indices.append(temp)
    
    return indices

___
Functions to generate Targets
___

In [3]:
def get_count(digitList, n):
    counter = 0

    for item in digitList:
        if item == n:
            counter += 1
    return counter


def generateTargets(n):
    targets = []

    while len(targets) != n:
        temp = random.randint(0,9)

        if get_count(targets, temp) < 1:
            targets.append(temp)
    
    return targets

___
Functions to generate Pseudo Output List || Stimuli, Lures, and Fillers still need to be inserted
___

In [4]:
def generateOutputList(targetList, targetIndices, totalStimuli):
    outputList = []

    for i in range(totalStimuli):
        outputList.append(-1)

    targetIndices.sort()

    counter = 0
    for i in range(totalStimuli):
        if counter == 5:
            break
        
        if i == targetIndices[counter]:
            outputList[i] = targetList[counter]
            counter += 1

    return outputList

___
Functions to generate Stimuli Indices and insert Stimuli in pseudo output list
___

In [5]:

def getStimuliIndices_InsertStimuli(givenList, targetIndices, whatBack):
    stimuliIndices = []

    for i in range(len(givenList)):  
        if i in targetIndices:
            givenList[i - whatBack] =  givenList[i]
            stimuliIndices.append(i - whatBack)
    return stimuliIndices

___
Functions to generate Lure Indices and insert Lures in pseudo output list
___

In [6]:
def get_lure_indices(level, targetIndices, stimuliIndices, whatBack, totalStimuli):

    ## Fix possible number of lures according to whatBack
    no_lures = 0
    
    if whatBack == 1:
        no_lures = 3
    elif whatBack == 2:
        no_lures = 4
    elif whatBack == 3:
        no_lures = 5

    lure_indices = [] ## For attached lures
    lure_stimuli_indices = [] ## For unattached lures

    collected_target_indices = []


    ###### Make n + 1 back (target attached) lures for 1-Back block #######
    if whatBack == 1:
        for item in targetIndices:
            if (item + 1 not in stimuliIndices) and (item + 2 not in stimuliIndices) and (item + 2) < 18:
                collected_target_indices.append(item)
                lure_indices.append(item + 2)

    ###### Make n - 1 back unattached lures for level 2 and 3 #####
    if level == 3:
        if whatBack == 2:
            for item in targetIndices:
                if (item + 1 not in stimuliIndices) and (item + 2 not in stimuliIndices) and (item + 2) < 22:
                    collected_target_indices.append(item)
                    lure_stimuli_indices.append(item + 1)

        if whatBack == 3:
            for item in targetIndices:
                if (item + 1 not in stimuliIndices) and (item + 2 not in stimuliIndices) and (item + 3 not in stimuliIndices) and (item + 2) < 27:
                    collected_target_indices.append(item)
                    lure_stimuli_indices.append(item + 1)


    ## Now insert the other required lures (these will be attached unfortunately)
    ## If whatBack = 1, then these lures will need to be fixed afterwards
    remaining_lures = no_lures - len(lure_stimuli_indices)

    while remaining_lures > len(lure_indices):

        item = targetIndices[random.randint(0, 4)]
        if item in collected_target_indices:
            continue
        else:
            collected_target_indices.append(item)


        if level == 1:
            lure_indices = []
            lure_stimuli_indices = []
            break
        
        if level == 2:
            if whatBack == 1:
                    if (item + 1 not in lure_indices) and (item + 1 not in targetIndices) and (item + 1 not in stimuliIndices) and ((item + 2) < totalStimuli):
                        lure_indices.append(item + 2) 
            else:
                if (item + 1 not in lure_indices) and (item + 1 not in targetIndices) and (item + 1 not in stimuliIndices):
                    lure_indices.append(item + 1)

        if level == 3:
            if whatBack == 1:
                if (item + 1 not in lure_indices) and (item + 1 not in targetIndices) and (item + 1 not in stimuliIndices) and ((item + 2) < totalStimuli):
                    lure_indices.append(item + 2)
            
            elif whatBack == 2:
                ## At whatBack 2, attached lures can only be at n + 1 back 
                if (item + 1 not in lure_indices) and (item + 1 not in targetIndices) and (item + 1 not in stimuliIndices):
                    lure_indices.append(item + 1)

            elif whatBack == 3:
                selector = random.randint(0, 1)

                if (selector == 0) and (item + 1 not in lure_indices) and (item + 1 not in targetIndices) and (item + 1 not in stimuliIndices):
                    lure_indices.append(item + 1)
                if (selector == 1) and (item - 1 not in lure_indices) and (item - 1 not in targetIndices) and (item - 1 not in stimuliIndices):
                    lure_indices.append(item - 1)
    
    if level == 1:
            lure_indices = []
            lure_stimuli_indices = []
            
    return lure_indices, lure_stimuli_indices


def insertLures(outputList, lure_stimuli_indices, lure_indices, targetList, whatBack, level):

    for itemx in lure_stimuli_indices:
        m = random.randint(0, 9)
        while m in targetList:
            m = random.randint(0, 9)
        
        outputList[itemx] = m
        outputList[itemx + whatBack - 1] = m


    for item in lure_indices:
        if level != 1 and whatBack == 1:
            if outputList[item - 2] in targetList:
                outputList[item] = outputList[item - 2]
        else:
            if outputList[item - 1] in targetList:
                outputList[item] = outputList[item - 1]
            
            elif outputList[item + 1] in targetList:
                outputList[item] = outputList[item + 1]
    
    return outputList

___
Fix lures for 1-Back
___

In [7]:
def fix_lure_mess1(outputList, lure_stimuli_indices, lure_indices, stimuliIndices, target_indices, whatBack):
    if whatBack != 1:
        return outputList
        
    x = outputList
    for luIndx in lure_indices:

        if ((luIndx - whatBack - 1) in target_indices) and (luIndx in stimuliIndices):
            x[luIndx] = x[luIndx - whatBack - 1]
            x[luIndx + whatBack] = x[luIndx]
    
    return x


def fix_lure_mess2(outputList, lure_stimuli_indices, lure_indices, stimuliIndices, target_indices, whatBack):
    
    if whatBack != 1:
        return outputList

    x = outputList
    for luIndx in lure_indices:
        if ((luIndx - whatBack - 1) in target_indices):
            x[luIndx] = outputList[luIndx - whatBack - 1]
    return x

___
Functions to generate and insert fillers into pseudo output list
___

In [8]:
def checkFillerForLureTarget(givenList, targetList, lureIndices, givenIndex, randomNumber, whatBack):
    result = True

    if randomNumber in targetList:
        result = False

    if givenList[givenIndex] == randomNumber:
        result = False

    if givenList[givenIndex - 1] == randomNumber:
        result = False

    if givenIndex - 2 >= 0:
        if givenList[givenIndex - 2] == randomNumber:
            result = False

    if givenIndex - 3 >= 0:
        if givenList[givenIndex - 3] == randomNumber:
            result = False
    
    if givenIndex - 4 >= 0:
        if givenList[givenIndex - 4] == randomNumber:
            result = False

    if givenIndex + 1 != len(givenList):

        if givenList[givenIndex + 1] == randomNumber:
            result = False
    
    if givenIndex + 2 < len(givenList):

        if givenList[givenIndex + 2] == randomNumber:
            result = False

    if givenIndex + 3 < len(givenList):

        if givenList[givenIndex + 3] == randomNumber:
            result = False       
    
    if givenIndex + 4 < len(givenList):

        if givenList[givenIndex + 4] == randomNumber:
            result = False       

    return result


def insertFillers(givenList, stimuliIndices, targetIndices, lure_stimuli_indices, lureIndices, targetList, whatBack):
    for i in range(len(givenList)):
        if (i not in stimuliIndices) and (i not in targetIndices) and (i not in lureIndices) and (i not in lure_stimuli_indices) and (givenList[i] == -1):
            temp = random.randint(0, 9)

            while checkFillerForLureTarget(givenList, targetList, lureIndices, i, temp, whatBack) == False:
                temp = random.randint(0, 9)
            
            givenList[i] = temp
    return givenList

___
Functions to generate final conditions list
- need just whatBack and Level as inputs
- Final list will have stimuli, targets, lures, and fillers appropriately placed
___

In [9]:
def create_conditions_file(whatBack, level):

    # Check n back condition
    if whatBack < 1 or whatBack > 3:
        print("whatBack has to be between 1 and 3")
        return []
    
    # Get total number of stimuli based on n-back condition
    total_stimuli = 18
    if whatBack == 2:
        total_stimuli = 23
    if whatBack == 3:
        total_stimuli = 28

    ## Generate a list of targets
    targetList = generateTargets(5)

    ## Generate indices of the target in the output list
    targetIndices = [] 
    
    while len(targetIndices) < 5:
        targetIndices = createTargetIndices(total_stimuli, 3, whatBack, level)
        random.seed(random.seed(time.time()))

    ## Generate the half done outputList
    outputList = generateOutputList(targetList, targetIndices, total_stimuli)

    ## Generate the stimuli indices and insert the stimuli
    stimuliIndices = getStimuliIndices_InsertStimuli(outputList, targetIndices, whatBack)

    ## Get Lure indices and generate lures
    lure_indices, lure_stimuli_indices = get_lure_indices(level, targetIndices, stimuliIndices, whatBack, total_stimuli)
    lure_indices.sort()
    
    ## Insert Lures
    outputList = insertLures(outputList, lure_stimuli_indices, lure_indices, targetList, whatBack, level)

    ## Fix 1-Back lure mess
    outputList = fix_lure_mess1(outputList, lure_stimuli_indices, lure_indices, stimuliIndices, targetIndices, whatBack)
    outputList = fix_lure_mess2(outputList, lure_stimuli_indices, lure_indices, stimuliIndices, targetIndices, whatBack)
    
    ## Insert Fillers
    outputList = insertFillers(outputList, stimuliIndices, targetIndices, lure_stimuli_indices, lure_indices, targetList, whatBack)

    ## Create dataframe
    conditions = []

    for i in range(total_stimuli):
        temp = {}

        lure_label = ""
        target_label = ""
        stimuli = outputList[i]
        corrAns = ""


        if i in lure_indices:
            lure_label = "lure"
            corrAns = "l"
        
        if i in targetIndices:
            target_label = "target"
            corrAns = "space"

        if i in stimuliIndices:
            target_label = "stimuli"
        
        if i - (whatBack - 1) in lure_stimuli_indices:
            lure_label = "lure"
            corrAns = "l"
        
        temp = {"Stimuli": stimuli, "isTarget": target_label, "isLure": lure_label, "corrAns": corrAns}
        conditions.append(temp)
    
    return pd.DataFrame(conditions)


___
Create the Block 1 Level 1 conditions file
- 18 stimuli total
- 5 stimuli are targets
___

In [10]:
random.seed(19)

In [11]:
def convert_digit_to_letter(digit):
    if digit == 0:
        return 'a'
    elif digit == 1:
        return 'b'
    elif digit == 2:
        return 'c'
    elif digit == 3:
        return 'd'
    elif digit == 4:
        return 'e'
    elif digit == 5:
        return 'f'
    elif digit == 6:
        return 'g'
    elif digit == 7:
        return 'h'
    elif digit == 8:
        return 'i'
    elif digit == 9:
        return 'j'
    

In [12]:
## Get wordList and make word Conditions file
path = "/Users/farhan/Desktop/Baycrest Documents/Madeline/Aphasia_STM_stim_generation/Syllable_Project/"

conditionsPath = "/Users/farhan/Desktop/Baycrest Documents/Madeline/NBack/conditions"
for session in range (1, 13):
    print("Generating session: ", session)
    for block in range(1, 4):
        whatBack = 0
        lureLevel = 0

        for level in range(1, 9):
            if level == 1:
                whatBack = 1
                lureLevel = 1
            elif level == 2:
                whatBack = 1
                lureLevel = 3
            elif level == 3:
                whatBack = 2
                lureLevel = 1
            elif level == 4:
                whatBack = 2
                lureLevel = 2
            elif level == 5:
                whatBack = 2
                lureLevel = 3
            elif level == 6:
                whatBack = 3
                lureLevel = 1
            elif level == 7:
                whatBack = 3
                lureLevel = 2
            elif level == 8:
                whatBack = 3
                lureLevel = 3
        
            wordPath = path + "wordLists/wordList" + str(session) + ".txt"
            nonWordPath = path + "nonWordLists/nonWordList" + str(session) + ".txt"

            wordDF = pd.read_csv(wordPath, sep='\t')
            nonWordDF = pd.read_csv(nonWordPath, sep='\t')


            
            baseStimuliDF = create_conditions_file(whatBack, lureLevel)
            baseStimuliDF.Stimuli = baseStimuliDF.Stimuli.astype(str)

            ## Load 6 copies of the digit dataframe for manipulation
            digits = baseStimuliDF.copy(deep=True)
            word = baseStimuliDF.copy(deep=True)
            nonWord = baseStimuliDF.copy(deep=True)

            soundWord = baseStimuliDF.copy(deep=True)
            soundNonWord = baseStimuliDF.copy(deep=True)
            soundDigit = baseStimuliDF.copy(deep=True)

            ## Make the Text dataframes
            for index, row in word.iterrows():
                word.at[index, "Stimuli"] = wordDF.iloc[int(row["Stimuli"])]["Word|s"]
            
            for index, row in nonWord.iterrows():
                nonWord.at[index, "Stimuli"] = nonWordDF.iloc[int(row["Stimuli"])]["nonWord"]

            ## Make the Audio dataframes
            for index, row in soundWord.iterrows():
                soundWord.at[index, "Stimuli"] = "audio_files/word_audio/" + "word"  + str(session) + convert_digit_to_letter(int(row["Stimuli"])) + ".wav"
            for index, row in soundNonWord.iterrows():
                soundNonWord.at[index, "Stimuli"] = "audio_files/nw_audio/" + "nonword"  + str(session) + convert_digit_to_letter(int(row["Stimuli"])) + ".wav"
            for index, row in soundDigit.iterrows():
                soundDigit.at[index, "Stimuli"] = "audio_files/digit_audio/" + "digit"  + str(session) + convert_digit_to_letter(int(row["Stimuli"])) + ".wav"

            ## Create destination directories names
            output_digit_dir_name = conditionsPath + "/" + "session_" + str(session) + "/" + "Digit" + "/" + "Block-" + str(block)
            output_word_dir_name = conditionsPath + "/" + "session_" + str(session) + "/" + "Word" + "/" + "Block-" + str(block)
            output_non_word_dir_name = conditionsPath + "/" + "session_" + str(session) + "/" + "nonWord" + "/" "Block-" + str(block)

            output_word_sound_dir_name = conditionsPath + "/" + "session_" + str(session) + "/" + "soundWord" + "/" + "Block-" + str(block)
            output_nw_sound_dir_name = conditionsPath + "/" + "session_" + str(session) + "/" + "soundNonWord" + "/" + "Block-" + str(block)
            output_digit_sound_dir_name = conditionsPath + "/" + "session_" + str(session) + "/" + "soundDigit" + "/" + "Block-" + str(block)


            ## Create proper destination directories if they dont exist
            os.makedirs(output_digit_dir_name, exist_ok=True)
            os.makedirs(output_word_dir_name, exist_ok=True)
            os.makedirs(output_non_word_dir_name, exist_ok=True)

            os.makedirs(output_word_sound_dir_name, exist_ok=True)
            os.makedirs(output_nw_sound_dir_name, exist_ok=True)
            os.makedirs(output_digit_sound_dir_name, exist_ok=True)

            ## Create proper destination directories
            digits_destination = os.path.join(output_digit_dir_name , "Level_" + str(level) + ".csv") 
            word_destination = os.path.join(output_word_dir_name , "Level_" + str(level) + ".csv") 
            non_word_destination = os.path.join(output_non_word_dir_name , "Level_" + str(level) + ".csv") 

            sound_word_destination = os.path.join(output_word_sound_dir_name , "Level_" + str(level) + ".csv")
            sound_nw_destination = os.path.join(output_nw_sound_dir_name , "Level_" + str(level) + ".csv") 
            sound_digit_destination = os.path.join(output_digit_sound_dir_name , "Level_" + str(level) + ".csv") 

            ## Create the CSV files
            digits.to_csv(digits_destination, index = False)
            word.to_csv(word_destination, index = False)
            nonWord.to_csv(non_word_destination, index = False)

            soundWord.to_csv(sound_word_destination, index = False)
            soundNonWord.to_csv(sound_nw_destination, index = False)
            soundDigit.to_csv(sound_digit_destination, index = False)

Generating session:  1
Generating session:  2
Generating session:  3
Generating session:  4
Generating session:  5
Generating session:  6
Generating session:  7
Generating session:  8
Generating session:  9
Generating session:  10
Generating session:  11
Generating session:  12


In [13]:
# import shutil

# m = "/Users/farhan/Desktop/Baycrest Documents/Madeline/farhanNBack/audio_files/kikos"
# for i in range(1, 9):
#     for filename in os.listdir(m):

#         new_name = "digit1"+ str(i) + filename[7] + ".wav"

#         source = os.path.join(m, filename) 

#         temp_destination = os.path.join("/Users/farhan/Desktop/Baycrest Documents/Madeline/farhanNBack/audio_files/kikos" , new_name) 
 
#         os.rename(source, temp_destination)

#         shutil.copy(temp_destination, "/Users/farhan/Desktop/Baycrest Documents/Madeline/farhanNBack/audio_files/digit_audio/" + new_name)